<a href="https://colab.research.google.com/github/GuanRuLai/Python-Project-Image-Generator-Webapp/blob/main/PetAvatar_webapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary libraries

In [1]:
!pip install torch diffusers pillow transformers sentencepiece gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00


# Import libraries

In [2]:
import torch
from diffusers import FluxPipeline
from PIL import Image
import gradio as gr

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

# Check if GPU accelerator is used

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. GPU will be used.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. CPU will be used.")

CUDA is available. GPU will be used.


# Load FLUX.1 [schnell] model for both mode

In [4]:
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell",
                                    torch_dtype=torch.bfloat16,
                                    use_fast=True)
pipe.enable_model_cpu_offload() # Save memory
pipe.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

Keyword arguments {'use_fast': True} are not expected by FluxPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
It seems like you have activated model offloading by calling `enable_model_cpu_offload`, but are now manually moving the pipeline to GPU. It is strongly recommended against doing so as memory gains from offloading are likely to be lost. Offloading automatically takes care of moving the individual components vae, text_encoder, text_encoder_2, tokenizer, tokenizer_2, transformer, scheduler to GPU when needed. To make sure offloading works as expected, you should consider moving the pipeline back to CPU: `pipeline.to('cpu')` or removing the move altogether if you use offloading.


FluxPipeline {
  "_class_name": "FluxPipeline",
  "_diffusers_version": "0.31.0",
  "_name_or_path": "black-forest-labs/FLUX.1-schnell",
  "scheduler": [
    "diffusers",
    "FlowMatchEulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "FluxTransformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

# Define function for image generation

In [54]:
def generate_images_from_text(prompt, num_images=2):
    """
    根據文本描述生成多張圖片
    :param prompt: 用戶輸入的描述
    :param num_images: 要生成的圖片數量
    :return: 生成的圖片列表
    """
    images = []
    for _ in range(num_images):
        image = pipe(prompt, num_inference_steps=10).images[0]
        images.append(image)

    return images

# Gradio API

In [59]:
def gradio_generate(prompt, num_images=2, previous_prompt=""):
    images = generate_images_from_text(prompt, num_images)

    return images

# Build gradio web app

In [63]:
with gr.Blocks() as demo:
    gr.Markdown("# Text generation image tool")

    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Enter image description",
                                placeholder="For example: a cat in a space suit with a starry sky in the background.")
            num_images = gr.Slider(label="Number of images generated",
                                   minimum=1, maximum=5, step=1, value=2)
            submit = gr.Button("Generate pictures")

        with gr.Column():
            output_gallery = gr.Gallery(label="Generated Images", columns=2, height="400px")

    submit.click(gradio_generate, inputs=[prompt, num_images], outputs=output_gallery)

In [64]:
if __name__ == "__main__":
    demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://de86af7311c39c7347.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/10 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7865 <> https://de86af7311c39c7347.gradio.live
